<a href="https://colab.research.google.com/github/LinaDanilina/recommender-system/blob/master/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
! ls
%cd drive/My Drive/spotlight

drive  sample_data
/content/drive/My Drive/spotlight


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

In [0]:
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    explained_variance_score,
    roc_auc_score,
    log_loss,
)

In [0]:
train=pd.read_csv('train.csv')
train=train[['user_id','movie_id','rating','unix_timestamp']]
train=train.rename(columns={'user_id':'user_ids','movie_id':'item_ids','rating':'ratings','unix_timestamp':'timestamps'})

test=pd.read_csv('test.csv',delimiter=',')
test=test[['user_id','movie_id','rating','unix_timestamp']]
test=test.rename(columns={'user_id':'user_ids','movie_id':'item_ids','rating':'ratings','unix_timestamp':'timestamps'})

In [0]:
from spotlight.interactions import Interactions

train=Interactions(train['user_ids'][:].values,train['item_ids'][:].values,train['ratings'][:].values,train['timestamps'][:].values)
test=Interactions(test['user_ids'][:].values,test['item_ids'][:].values,test['ratings'][:].values,test['timestamps'][:].values)
print(train,test)

<Interactions dataset (943 users x 1682 items x 75000 interactions)> <Interactions dataset (943 users x 1680 items x 25000 interactions)>


In [0]:
train.user_ids

array([ 46, 845,  22, ..., 523,   4, 406])

We can feed our dataset to the ExplicitFactorizationModel class - and sklearn-like object that allows us to train and evaluate the explicit factorization models.

Internally, the model uses the BilinearNet class to represents users and items. It's composed of a 4 embedding layers:

* a (num_users x latent_dim) embedding layer to represent users,
* a (num_items x latent_dim) embedding layer to represent items,
* a (num_users x 1) embedding layer to represent user biases, and
* a (num_items x 1) embedding layer to represent item biases.

Together, these give us the predictions. Their accuracy is evaluated using one of the Spotlight losses. In this case, we'll use the regression loss, which is simply the squared difference between the true and the predicted rating.

In [0]:
import torch

from spotlight.factorization.explicit import ExplicitFactorizationModel

model = ExplicitFactorizationModel(loss='regression',
                                   embedding_dim=128,  # latent dimensionality
                                   n_iter=10,  # number of epochs of training
                                   batch_size=1024,  # minibatch size
                                   l2=1e-9,  # strength of L2 regularization
                                   learning_rate=1e-3,
                                   use_cuda=torch.cuda.is_available())

In [0]:
start_time = time.time()
history=model.fit(train, verbose=True)
train_time = time.time() - start_time

Epoch 0: loss 13.194399627479347
Epoch 1: loss 8.196673960299105
Epoch 2: loss 2.0793311853666565
Epoch 3: loss 1.119748984639709
Epoch 4: loss 0.9594384278800037
Epoch 5: loss 0.9009330546533739
Epoch 6: loss 0.8742705905759657
Epoch 7: loss 0.8572772322474299
Epoch 8: loss 0.8460370205544137
Epoch 9: loss 0.8336294865286028


In [0]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(model, train)
test_rmse = rmse_score(model, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.899, test RMSE 0.947


In [0]:
start_time = time.time()
predictions = model.predict(test.user_ids, test.item_ids)
test_time = time.time() - start_time

In [0]:
def metrics(data_true, data_pred,time_train, time_test):
    mse=np.sqrt(mean_squared_error(data_true, data_pred))
    mae=mean_absolute_error(data_true, data_pred)
    r2=r2_score(data_true, data_pred)
    ex_var=explained_variance_score(test.ratings, predictions)
    df=pd.DataFrame({"RMSE": mse, "MAE":mae, "R2_score":r2, "explained variance":ex_var,'train time':time_train,
                     'test time':time_test},index=[0])
    return df

In [0]:
metrics(test.ratings, predictions, train_time, test_time)

,RMSE,MAE,R2_score,explained variance,train time,test time
0,0.946716,0.745732,0.297497,0.297691,9.38648,0.01899
